# Descripción del proyecto

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

- Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
- Crear un modelo para predecir el volumen de reservas en pozos nuevos.
- Elegir los pozos petrolíferos que tienen los valores estimados más altos.
- Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

## Condiciones:

- Solo se debe usar la regresión lineal para el entrenamiento del modelo.
- Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
- El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
- Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

Los datos son sintéticos: los detalles del contrato y las características del pozo no se publican.

## Descripción de datos

Los datos de exploración geológica de las tres regiones se almacenan en archivos:

- geo_data_0.csv. 
- geo_data_1.csv. 
- geo_data_2.csv. 
- id — identificador único de pozo de petróleo
- f0, f1, f2 — tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas)
- product — volumen de reservas en el pozo de petróleo (miles de barriles).

## Instrucciones del proyecto

1. Descarga y prepara los datos. Explica el procedimiento.
2. Entrena y prueba el modelo para cada región en geo_data_0.csv:

    1. Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25

    2. Entrena el modelo y haz predicciones para el conjunto de validación.

    3. Guarda las predicciones y las respuestas correctas para el conjunto de validación.

    4. Muestra el volumen medio de reservas predicho y RMSE del modelo.

    5. Analiza los resultados.

    6. Coloca todos los pasos previos en funciones, realiza y ejecuta los pasos 2.1-2.5 para los archivos 'geo_data_1.csv' y 'geo_data_2.csv'.

3. Prepárate para el cálculo de ganancias:

    1. Almacena todos los valores necesarios para los cálculos en variables separadas.

    2. Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.

    3. Presenta conclusiones sobre cómo preparar el paso para calcular el beneficio.

4. Escribe una función para calcular la ganancia de un conjunto de pozos de petróleo seleccionados y modela las predicciones:

    1. Elige los 200 pozos con los valores de predicción más altos de cada una de las 3 regiones (es decir, archivos 'csv').

    2. Resume el volumen objetivo de reservas según dichas predicciones. Almacena las predicciones para los 200 pozos para cada una de las 3 regiones.

    3. Calcula la ganancia potencial de los 200 pozos principales por región. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección.

5. Calcula riesgos y ganancias para cada región:

    1. Utilizando las predicciones que almacenaste en el paso 4.2, emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.

    2. Encuentra el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdidas. La pérdida es una ganancia negativa, calcúlala como una probabilidad y luego exprésala como un porcentaje.

    3. Presenta tus conclusiones: propón una región para el desarrollo de pozos petrolíferos y justifica tu elección. ¿Coincide tu elección con la elección anterior en el punto 4.3?

# Librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from pprint import pprint
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import KNNImputer
from sklearn.utils import shuffle
import re

# Carga de datos

In [2]:
data_1 = pd.read_csv('datasets/geo_data_0.csv')
data_2 = pd.read_csv('datasets/geo_data_1.csv')
data_3 = pd.read_csv('datasets/geo_data_2.csv')

In [3]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
print(data_1.info())
print()
print(data_2.info())
print()
print(data_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

Podemos identificar que los datos son netamente numéricos a excepción del contrato que identifica el pozo de petróleo y que no tenemos datos ausentes. Los tipos de datos están bien, por lo que vamos a dejarlos tal como están.

# Describe

In [7]:
print(data_1.describe())
print()
print(data_2.describe())
print()
print(data_3.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%      

Las estadísticas descriptivas de las variables f0, f1, y f2 no nos dicen nada, sin embargo, podemos ver el volumen medio de reservas de petroleo de cada región, en la región 1 es de 92.5 miles de barriles, para la region 2 es 68.82 miles de barriles y para la region 3 es de 95 mil barriles.

# Limpieza de datos

In [8]:
#Calculamos los ausentes
print('Ausentes: \n',data_1.isna().sum())
print('Ausentes: \n',data_2.isna().sum())
print('Ausentes: \n',data_3.isna().sum())

Ausentes: 
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Ausentes: 
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
Ausentes: 
 id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [34]:
#Calculamos los pozos con valores negativos
print('Pozos con reservas negativas de la región 1: \n',data_1[data_1['product']<0]['product'].count())
print('Pozos con reservas negativas de la región 2: \n',data_2[data_2['product']<0]['product'].count())
print('Pozos con reservas negativas de la región 3: \n',data_3[data_3['product']<0]['product'].count())

Pozos con reservas negativas: 
 0
Pozos con reservas negativas: 
 0
Pozos con reservas negativas: 
 0


No tenemos pozos con valores negativos, lo cual está bien.

In [35]:
#Calculamos los pozos con valores negativos
print('Pozos con 0 reservas de la región 1: \n',data_1[data_1['product']==0]['product'].count())
print('Pozos con 0 reservas de la región 2: \n',data_2[data_2['product']==0]['product'].count())
print('Pozos con 0 reservas de la región 3: \n',data_3[data_3['product']==0]['product'].count())

Pozos con 0 reservas de la región 1: 
 1
Pozos con 0 reservas de la región 2: 
 8235
Pozos con 0 reservas de la región 3: 
 1


Podemos ver la cantidad de pozos que tienen 0 barriles de petroleo es de solo 1 en la región uno, 8235 en la región 2 y 1 en la región 3.

In [37]:
#Calculamos los pozos con valores negativos
print('Proporción de la región 1: \n',100*data_1[data_1['product']==0]['product'].count()/data_1.shape[0])
print('Proporción de la región 2: \n',100*data_2[data_2['product']==0]['product'].count()/data_2.shape[0])
print('Proporción de la región 3: \n',100*data_3[data_3['product']==0]['product'].count()/data_3.shape[0])

Proporción de la región 1: 
 0.001
Proporción de la región 2: 
 8.235
Proporción de la región 3: 
 0.001


El porcentaje medianamente significativo es el 8% de la región 2.

In [38]:
print('Duplicados de la región 1: \n',data_1.duplicated().sum())
print('Duplicados de la región 2: \n',data_2.duplicated().sum())
print('Duplicados de la región 3: \n',data_3.duplicated().sum())

Duplicados de la región 1: 
 0
Duplicados de la región 2: 
 0
Duplicados de la región 3: 
 0


No tenemos valores duplicados o pozos duplicados, por lo que podemos proseguir con nuestro algoritmo.

# Entrenamiento de la región 1

In [39]:
seed=12345
df_train, df_valid=train_test_split(data_1,test_size=0.25,random_state=seed)